In [4]:
from functools import reduce  # for py3
import numpy as np
from matplotlib import pyplot as plt

class Perceptron(object):
    def __init__(self, input_num, activator):
        '''
        初始化感知器，设置输入参数的个数，以及激活函数。
        激活函数的类型为float -> float
        '''
        self.activator = activator
        # 权重向量初始化为0
        self.weights = [0.0 for _ in range(input_num)]
        # 偏置项初始化为0
        self.bias = 0.0
        self.learningCurve=[]
    def __str__(self):
        '''
        打印学习到的权重、偏置项
        '''
        return 'weights\t:%s\nbias\t:%f\n' % (self.weights, self.bias)


    def predict(self, input_vec):
        '''
        输入向量，输出感知器的计算结果
        '''
        # 把input_vec[x1,x2,x3...]和weights[w1,w2,w3,...]打包在一起
        # 变成[(x1,w1),(x2,w2),(x3,w3),...]
        # 然后利用map函数计算[x1*w1, x2*w2, x3*w3]
        # 最后利用reduce求和

        #list1 = list(self.weights)
        #print ("predict self.weights:", list1)

        
        return self.activator(
            reduce(lambda a, b: a + b,
                   list(map(lambda tp: tp[0] * tp[1],
                       zip(input_vec, self.weights)))
                , 0.0) + self.bias)
    def train(self, input_vecs, labels, iteration, rate):
        '''
        输入训练数据：一组向量、与每个向量对应的label；以及训练轮数、学习率
        '''
        self.learningCurve=[]
        for i in range(iteration):
            error = self._one_iteration(input_vecs, labels, rate)
            self.learningCurve.append(error)
            

    def _one_iteration(self, input_vecs, labels, rate):
        '''
        一次迭代，把所有的训练数据过一遍
        '''
        # 把输入和输出打包在一起，成为样本的列表[(input_vec, label), ...]
        # 而每个训练样本是(input_vec, label)
        samples = zip(input_vecs, labels)
        
        # 对每个样本，按照感知器规则更新权重
        for (input_vec, label) in samples:
            # 计算感知器在当前权重下的输出
            output = self.predict(input_vec)
            # 更新权重
            self._update_weights(input_vec, output, label, rate)

        # cost
        predicts = list(map(self.predict,input_vecs))
        return sum(predicts!=labels)

    def _update_weights(self, input_vec, output, label, rate):
        '''
        按照感知器规则更新权重
        '''
        # 把input_vec[x1,x2,x3,...]和weights[w1,w2,w3,...]打包在一起
        # 变成[(x1,w1),(x2,w2),(x3,w3),...]
        # 然后利用感知器规则更新权重
        delta = label - output# 0 1 -1
        self.weights = list(map( lambda tp: tp[1] + rate * delta * tp[0], zip(input_vec, self.weights)) )

        # 更新bias
        self.bias += rate * delta
        '''
        print("_update_weights() -------------")
        print("label - output = delta:" ,label, output, delta)
        print("weights ", self.weights)
        print("bias", self.bias)
        '''
    def drawLearningCurve(self,sigma,fname='learningCurve.png'):
         plt.plot(range(len(self.learningCurve)),self.learningCurve)
         plt.xlim(0, len(self.learningCurve))
         plt.xlabel('epoch')
         plt.ylabel('Cost')
         plt.title('LearningCurve (sigma={})'.format(sigma))
         plt.savefig(fname)
         #plt.show()
         plt.close()



def f(x):
    '''
    定义激活函数f sign(x)
    '''
    return 1 if x > 0 else 0


def get_training_dataset(sigma=0.01):
    '''
    https://blog.csdn.net/mmh19891113/article/details/82114815
    （1）生成数据：正样本样本服从高斯分布，均值为[1,1]，协方差矩阵为单位矩阵；
    负样本服从三个高斯分布的混合高斯分布，这三个高斯分布的均值分别为[0,0], [0,1], [1,0]，
    每个高斯分布的协方差矩阵均为(sigma*单位矩阵)。
    '''
    # 构建训练数据
    # 输入向量
    # 正样例
    mu = np.array([[1, 1]])
    Sigma = np.array([[1, 0], [0, 1]])
    Sigma2 = sigma*Sigma 
    positive = np.dot(np.random.randn(300, 2), Sigma2) + mu
    avg = np.array([[0, 0]])
    print(np.dot(np.random.randn(100, 2), Sigma2) + avg)
    negetive = list(np.dot(np.random.randn(100, 2), Sigma2) + avg)
    avg = np.array([[0, 1]])
    negetive.extend(list(np.dot(np.random.randn(100, 2), Sigma2) + avg))
    avg = np.array([[1, 0]])
    negetive.extend(list(np.dot(np.random.randn(100, 2), Sigma2) + avg))
    negetive = np.array(negetive)
    input_vecs = np.concatenate((positive,negetive),axis=0)
    # 期望的输出列表
    labels = np.zeros(600,dtype=int)
    labels[:300] = np.ones(300,dtype=int)
    labels[300:] = np.zeros(300,dtype=int)
    return input_vecs, labels 
    
def train_perceptron(input_vecs, labels, niter=100,rate=0.1):
    '''
    使用and真值表训练感知器
    '''
    # 创建感知器，输入参数个数为2（因为and是二元函数），激活函数为f
    p = Perceptron(2, f)
    # 训练，迭代10轮, 学习速率为0.1
    p.train(input_vecs, labels, niter, rate)
    #返回训练好的感知器
    return p

def drawscatter(input_vecs,labels,perceptron,sigma,fname='scatter_line.png'):
    input_vecs_generator=iter(input_vecs)
    colors='rb'
    for label in labels:
        xs = next(input_vecs_generator)
        plt.scatter(xs[0],xs[1],marker='x',c=colors[label])
    
    line_x = np.arange(-0.1,1.5,0.5)
    # weights*[x0,x1]+bias = 0
    line_y = -(line_x*perceptron.weights[0]+perceptron.bias)/perceptron.weights[1]
    plt.plot(line_x,line_y)

    plt.title('scatter and line (sigma={})'.format(sigma))
    plt.savefig(fname)
    #plt.show()
    plt.close()

if __name__ == '__main__': 
    input_vecs, labels = get_training_dataset(sigma=0.01)
    and_perceptron = train_perceptron(input_vecs, labels, niter=10,rate=0.1)
    # Learning Curve
    and_perceptron.drawLearningCurve(sigma=0.01)
    # scatter
    drawscatter(input_vecs, labels, and_perceptron,sigma=0.01)

[[-0.01328622 -0.00983598]
 [ 0.00069705 -0.00764115]
 [-0.01218894 -0.00046477]
 [ 0.00691112 -0.0257893 ]
 [-0.00283325 -0.00188227]
 [ 0.00477458  0.00796023]
 [-0.01024851 -0.0084813 ]
 [-0.01557858  0.00759439]
 [ 0.00031871  0.00953231]
 [-0.01080631 -0.00136321]
 [ 0.00344869  0.00492402]
 [ 0.00162073  0.00964332]
 [-0.00743474 -0.00843176]
 [ 0.01153031  0.01914287]
 [-0.00330311  0.00593901]
 [ 0.01089882 -0.00078215]
 [ 0.00849551  0.00067028]
 [ 0.00088354  0.01120628]
 [-0.0007873  -0.00194067]
 [-0.0044493  -0.00244599]
 [-0.0049682  -0.00679641]
 [ 0.00951545 -0.00658729]
 [-0.00817903 -0.02254946]
 [-0.02177675 -0.00612029]
 [-0.00398557 -0.00441703]
 [-0.01420526 -0.00339717]
 [ 0.00886033 -0.00348316]
 [ 0.00676418  0.00442314]
 [-0.00644026  0.00411727]
 [-0.00117964 -0.00337255]
 [-0.0143789   0.00367457]
 [ 0.01556171  0.01067385]
 [ 0.0046937  -0.0109882 ]
 [ 0.00530211 -0.00919707]
 [ 0.00426053  0.00178143]
 [ 0.00085094 -0.00412407]
 [ 0.00861879 -0.0076563 ]
 

[[ 0.01842193  0.07835063]
 [ 0.05251683  0.08159355]
 [ 0.1048097   0.1406474 ]
 [-0.06876921 -0.02343194]
 [-0.03725162 -0.11799651]
 [ 0.14142653 -0.03548278]
 [ 0.13864883  0.09842923]
 [-0.16048374  0.07179275]
 [ 0.0577642   0.03114916]
 [ 0.10158278  0.0212844 ]
 [ 0.08984923  0.17120031]
 [-0.16173319 -0.05298228]
 [-0.08517304 -0.20606885]
 [ 0.1257752  -0.08089272]
 [ 0.11706578  0.10443364]
 [ 0.08034159  0.09434537]
 [ 0.15502228  0.10286425]
 [ 0.06373581  0.08311772]
 [ 0.12345452  0.03460589]
 [-0.03891901  0.12693658]
 [ 0.18554303  0.00991529]
 [ 0.01258151 -0.0791193 ]
 [ 0.03336138  0.12741026]
 [-0.14710437  0.02122115]
 [-0.29957466  0.17918114]
 [-0.05400348  0.10934977]
 [ 0.08024591  0.21046333]
 [ 0.08432157  0.00823906]
 [-0.0313748  -0.02614413]
 [ 0.02326387  0.12352618]
 [-0.11490717  0.1253628 ]
 [ 0.03227694  0.04057143]
 [ 0.06275138  0.06954074]
 [-0.1124725   0.00134296]
 [-0.09365397 -0.05699256]
 [-0.13782042  0.09937843]
 [ 0.01316002  0.03783448]
 

In [ ]:
for i,sigma in enumerate([0.01, 0.05, 0.1, 0.5, 1.0]):
    input_vecs, labels = get_training_dataset(sigma=sigma)
    and_perceptron = train_perceptron(input_vecs, labels, niter=20,rate=0.1)
    # Learning Curve
    and_perceptron.drawLearningCurve(sigma=sigma,fname='learningCurve_{}.png'.format(i))
    # scatter
    drawscatter(input_vecs, labels, and_perceptron, sigma=sigma, fname='scatter_line_{}.png'.format(i))